In [0]:
import nltk
import pandas as pd

In [0]:
dataset = pd.read_csv('SMSSpamCollection.txt', delimiter='\t', names=['label', 'message'])

In [42]:
dataser.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [34]:
dataset.shape

(5572, 2)

In [35]:
dataset.groupby('label').describe()

messages                                                               
         count unique                                                top freq
label                                                                        
ham       4825   4516                             Sorry, I'll call later   30
spam       747    653  Please call our customer service representativ...    4

In [48]:
Y = dataset['label']
Y.head()

0     ham
1     ham
2    spam
3     ham
4     ham
Name: label, dtype: object

In [8]:
nltk.download('all')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

In [0]:
def split_tokens(message):
  message = message.lower()
  word_tokens = word_tokenize(message)
  return word_tokens

In [0]:
dataset['tokenized_message'] = dataset.apply(lambda row: split_tokens(row['message']),axis=1)

In [52]:
dataset.head()

,label,message,tokenized_message
0,ham,"Go until jurong point, crazy.. Available only ...","[go, until, jurong, point, ,, crazy.., availab..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar, ..., joking, wif, u, oni, ...]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,ham,U dun say so early hor... U c already then say...,"[u, dun, say, so, early, hor, ..., u, c, alrea..."
4,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, i, do, n't, think, he, goes, to, usf, ,,..."


In [0]:
from nltk.stem.wordnet import WordNetLemmatizer

In [0]:
def split_into_lemmas(message):
  lemma=[]
  lemmatizer = WordNetLemmatizer()
  for word in message:
    a = lemmatizer.lemmatize(word)
    lemma.append(a)
  return lemma

In [0]:
dataset['lemmatized_message'] = dataset.apply(lambda row: split_into_lemmas(row['tokenized_message']), axis=1)

In [58]:
dataset.head()

,label,message,tokenized_message,lemmatized_message
0,ham,"Go until jurong point, crazy.. Available only ...","[go, until, jurong, point, ,, crazy.., availab...","[go, until, jurong, point, ,, crazy.., availab..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar, ..., joking, wif, u, oni, ...]","[ok, lar, ..., joking, wif, u, oni, ...]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,ham,U dun say so early hor... U c already then say...,"[u, dun, say, so, early, hor, ..., u, c, alrea...","[u, dun, say, so, early, hor, ..., u, c, alrea..."
4,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, i, do, n't, think, he, goes, to, usf, ,,...","[nah, i, do, n't, think, he, go, to, usf, ,, h..."


In [0]:
from nltk.corpus import stopwords

In [0]:
def stopword_removal(message):

    stop_words = set(stopwords.words('english'))

    filtered_sentence = []

    filtered_sentence = ' '.join([word for word in message if word not in stop_words])

    return filtered_sentence

In [0]:
dataset['preprocessed_message'] = dataset.apply(lambda row: stopword_removal(row['lemmatized_message']), axis=1)

In [66]:
dataset.head()

,label,message,tokenized_message,lemmatized_message,preprocessed_message
0,ham,"Go until jurong point, crazy.. Available only ...","[go, until, jurong, point, ,, crazy.., availab...","[go, until, jurong, point, ,, crazy.., availab...","go jurong point , crazy.. available bugis n gr..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar, ..., joking, wif, u, oni, ...]","[ok, lar, ..., joking, wif, u, oni, ...]",ok lar ... joking wif u oni ...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, in, 2, a, wkly, comp, to, win, f...",free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,"[u, dun, say, so, early, hor, ..., u, c, alrea...","[u, dun, say, so, early, hor, ..., u, c, alrea...",u dun say early hor ... u c already say ...
4,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, i, do, n't, think, he, goes, to, usf, ,,...","[nah, i, do, n't, think, he, go, to, usf, ,, h...","nah n't think go usf , life around though"


In [71]:
X = dataset['preprocessed_message']
X.head()

0    go jurong point , crazy.. available bugis n gr...
1                      ok lar ... joking wif u oni ...
2    free entry 2 wkly comp win fa cup final tkts 2...
3          u dun say early hor ... u c already say ...
4            nah n't think go usf , life around though
Name: preprocessed_message, dtype: object

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

tf_vectorizer = CountVectorizer(ngram_range=(1, 2),min_df = (1/len(Y)), max_df = 0.7)

#tf_vectorizer = CountVectorizer(ngram_range=(1, 2))

Total_Dictionary_TDM = tf_vectorizer.fit(X)

message_data_TDM = Total_Dictionary_TDM.transform(X)

In [84]:
message_data_TDM[1]
message_data_TDM.shape

(5572, 40377)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2),min_df = (1/len(Y)), max_df = 0.7)

Total_Dictionary_TFIDF = tfidf_vectorizer.fit(X)

message_data_TFIDF = Total_Dictionary_TFIDF.transform(X)

In [83]:
message_data_TFIDF[1]
message_data_TFIDF.shape

(5572, 40377)

In [0]:
from sklearn.model_selection import train_test_split  #Splitting the data for training and testing
seed = 10
X_train,X_test, Y_train, Y_test = train_test_split(message_data_TDM, Y, test_size=.1)

In [110]:
from sklearn.ensemble import RandomForestClassifier

rfclassifier = RandomForestClassifier(max_depth=5, n_estimators=15, max_features=60, random_state=10)

rfclassifier = rfclassifier.fit(X_train, Y_train)


rfscore=rfclassifier.score(X_test, Y_test)

print('Random Forest classification after model tuning',rfscore)

Random Forest classification after model tuning 0.8924731182795699


In [111]:
from sklearn.svm import SVC

svcclassifier = SVC(kernel="linear", C=10,random_state=10)

svcclassifier = svcclassifier.fit(X_train, Y_train)

score = svcclassifier.score(X_test, Y_test)

print('SVM Classifier : ',score)

SVM Classifier :  0.9838709677419355
